# Section 0

QPM: Assignment 8

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: We did not understand what the first sentence of the assignment was referring to (Out-of-sample and in-sample). We only used the given dates to limit the window over which we compute the Sharpe ratio.

# Section 0.bis

We retrieve the factors excess returns from the excel file and store it in the dataframe ```factors```.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# Import the data from the xlsx file
factors = pd.read_excel('/Users/ilyas/Library/CloudStorage/OneDrive-Personnel/EDHEC/M2/Quantitative Portfolio Management/Assignments/QPM-FactorsData.xlsx')
factors['Dates'] = pd.to_datetime(factors['Dates'], format='%Y%m')
factors = factors.set_index(factors.Dates)
factors = factors.drop('Dates',axis=1)

factors.head()

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
1967-02-01,0.0078,0.0334,-0.0217,0.0194,-0.0094,0.0356,0.035317,-0.002064,0.0262
1967-03-01,0.0399,0.0163,0.0031,0.0090,-0.0151,0.0142,0.018876,-0.016933,0.0081
1967-04-01,0.0389,0.0062,-0.0264,0.0243,-0.0375,0.0064,0.010983,-0.029519,0.0171
1967-05-01,-0.0433,0.0198,0.0080,-0.0175,0.0161,0.0067,0.005234,0.024686,0.0201
1967-06-01,0.0241,0.0596,0.0096,-0.0064,-0.0239,0.0603,0.002945,-0.021700,-0.0163


# Section 1

We compute the volatility timed-factors: the returns at time t+1 is divided by the variance of the returns from t-12 until t. We then multiply it by the constant ```c```.
We store these new factors in the dataframe ```vol_factors```.

In [2]:
# Initialisation
vol_window = 12
vol_factors = np.zeros((len(factors)-vol_window-1,9))

# Loop over the factor returns and divide the factor at time t+1 by the volatility from t-12 until t
for i in range(vol_window,len(factors)-1):
    last12_vol = factors.iloc[i-vol_window:i].std()**2
    vol_factors[i-vol_window] = factors.iloc[i]/last12_vol
    
# Attempt for c, but because we are unsure, we are leaving it to 1
c = np.array(factors.std()/np.std(vol_factors,axis=0))

c = 1
vol_factors = c*vol_factors

# Store the results in a DataFrame
vol_factors = pd.DataFrame(vol_factors)
vol_factors.columns=factors.columns.values
vol_factors = vol_factors.set_index(factors.index[vol_window+1:])
vol_factors.head()

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
1968-03-01,-35.835405,-79.610803,19.666910,-4.044894,30.851335,-42.884860,-12.692406,45.012596,43.503960
1968-04-01,1.638609,-20.714932,-10.435006,37.539788,-13.407313,34.671484,73.617569,-16.747000,-32.695174
1968-05-01,82.601569,80.350450,-18.097885,87.964436,-43.147938,53.394791,169.289201,-39.417906,2.998469
1968-06-01,13.646121,78.785598,16.406748,11.653967,-21.616400,33.507828,62.139523,-36.077366,31.249673
1968-07-01,4.765466,-1.776469,13.074556,-43.396924,31.584127,-16.582876,-14.456431,38.254984,119.409491


For each factor, we compute the mean-variance weights of a portfolio consisting of the original factor and the volatility-timed factor. 
We set the risk-free rate to 0, which means that our portfolio is actually equal to the tangent portfolio.
We store the weights in the list ```wgt```.

In [3]:
nb_col = len(factors.columns)
wgt = []

# Loop over each column
for col in range(nb_col):
    
    # Define a column vector of ones
    ones = np.ones((2, 1))
    
    # Define the gross risk-free rate
    Rf = 1.00
    
    # Column vector of gross mean returns from the time and not-timed factor
    mu = np.array(vol_factors.iloc[:,col].mean(),factors.iloc[vol_window+1:,col].mean())

    # Variance-covariance matrix of the time and not-timed factor
    V = np.cov(vol_factors.iloc[:,col],factors.iloc[vol_window+1:,col])

    # Inverse of the variance-covariance matrix
    Vinverse = np.linalg.inv(V)

    # Mean-variance weights for gamma = 5
    gamma = 5
    w_mv = (1/gamma) * Vinverse @ (mu-Rf*ones)
    
    # Weight in risk-free asset (which will be 0 since Rf is 0)
    w_f = 1 - ones.T @ w_mv
    
    # Weights of tangency portfolio
    w_Tangency = w_mv/(ones.T @ w_mv)
    
    # Store the weights in the list wgt
    wgt.append(w_Tangency)



# Section 2

For each factor, we multiply the returns of the original series and of the volatility-timed series by the weights computed in question one and store the returns in the dataframe ```ret_port```.
We then compute the annualised Sharpe ratio of all the factors and display the results.

In [4]:
ret_port = np.zeros((len(vol_factors),nb_col))

# Compute the returns for each factor portfolio
for i in range(nb_col):
    ret_port[:,i] = wgt[i][0] * vol_factors.iloc[:,col] + wgt[i][1] * factors.iloc[vol_window+1:,col]

# Store the results in a DataFrame
ret_port = pd.DataFrame(ret_port)
ret_port.columns=factors.columns.values
ret_port = ret_port.set_index(factors.index[vol_window+1:])

# Compute the Sharpe ratio of the portfolio with the timed factors for the relevant period
ret_period = ret_port.loc['1970-01-01':]
Sharpe_Ratio_timed = np.sqrt(12) * ret_period.mean() / ret_period.std()
Sharpe_Ratio_timed

Market    0.841936
SMB       0.835109
HML       0.736191
RMW       0.824694
CMA       0.821410
UMD       0.838761
ROE       0.819150
IA        0.828982
BAB       0.765557
dtype: float64

We compute the Sharpe ratio of the initial factors return.

In [5]:
# Compute the Sharpe ratio of the factor returns only
fact_period = factors.loc['1970-01-01':]
Sharpe_Ratio_reg = np.sqrt(12) * fact_period.mean() / fact_period.std()
Sharpe_Ratio_reg

Market    0.446749
SMB       0.147819
HML       0.296244
RMW       0.419432
CMA       0.515325
UMD       0.480613
ROE       0.654944
IA        0.646213
BAB       0.872135
dtype: float64

# Section 3

Using a portfolio formed by the timed factor and the regular factor significantly increase the Sharpe ratio of said factor. This has been the case for every single factor, except BAB. The Sharpe ratio was sometimes multiplied by 2, or 3, such as for SMB, HML, UMD...

# Section 4

It is unfortunately very difficult to implement this approach in practice because it uses data that is not necessarily available at the time of the investment: we indeed used the whole time series to construct the mean-variance portfolios.
This analysis also omitted transaction costs.